<a href="https://colab.research.google.com/github/anothermartz/cog-Wav2Lip/blob/master/Easy_Wav2Lip_Fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a super early version of the optimized code as found here: https://github.com/devxpy/cog-Wav2Lip

In [ ]:
#@title <h1>Step 1: Setup
#@markdown 👈 Click that little circle play button first - it will ask for Google Drive access: <br>
#@markdown > Accept if your files are on Google Drive (recommended).
#@markdown <br> Alternatively, you can click deny and upload files manually, but this is slower.
#check if already installed
try:
  from google.colab import drive
  drive.mount('/content/drive')
except:
  print("google drive not linked")
#import tensorflow as tf
#import torch
#import sys
#print("Checking GPU is enabled:")
#if not tf.test.gpu_device_name():
#    sys.exit('No GPU in runtime. Please go to the "Runtime" menu, "Change runtime type" and select "GPU".')
#else:
#  gpu_name = torch.cuda.get_device_name(0)
#  gpu_name = gpu_name.replace(' ', '_')
#  print(f'GPU is {gpu_name}')

import re
giturl = 'https://github.com/anothermartz/cog-Wav2Lip.git'
#clone git in a way that allows me to change it very easily
!git clone {giturl}
regex = r'([^\/]+)(?=\.git)'
match = re.search(regex, giturl)
project_dir = match.group(1)
%cd '{project_dir}'
!pip install batch_face
!wget 'https://github.com/anothermartz/Easy-Wav2Lip/releases/download/Prerequesits/Wav2Lip.pth' -O 'checkpoints/Wav2Lip.pth'
from easy_functions import get_video_details
import os
from base64 import b64encode
from IPython.display import HTML, Audio, clear_output
import time
#!git clone https://github.com/anothermartz/GFPGAN.git

def show_video(file_path):
  """Function to display video in Colab"""
  mp4 = open(file_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  display(HTML("""
  <video controls width=600>
      <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    
    if hours > 0:
        return f'{hours}h {minutes}m {seconds}s'
    elif minutes > 0:
        return f'{minutes}m {seconds}s'
    else:
        return f'{seconds}s'

clear_output()
print("installation complete")

In [3]:
############################## user inputs #####################################
#@markdown <h1>Step 2: Select & process:</h1>

#@markdown On destktop: <h1></h1>Click the folder icon ( 📁 ) at the left edge of colab, find your file(s), right click, copy path, paste below:
#@markdown<br></br>
#@markdown On mobile: <h1></h1>Tap the hamburger button ( ☰ ) at the top left, click show file browser, long tab (hold) on Easy-Wav2Lip, upload, select your file(s), find them in the file browser, copy path, paste below:
video_or_image = "/content/drive/Othercomputers/Shadow/OfflineProjects/penguinz0/penguinz01.mp4" #@param {type:"string"}
vocal_track = "/content/drive/Othercomputers/Shadow/OfflineProjects/penguinz0/penguinz01.wav" #@param {type:"string"}
#@markdown ### [Padding:](https://github.com/anothermartz/Easy-Wav2Lip#tweak-padding)</h1> (Up, Down, Left, Right) <br>
U = 0 #@param {type:"slider", min:-40, max:100, step:5}
D = 10 #@param {type:"slider", min:-40, max:100, step:5}
L = 0 #@param {type:"slider", min:-40, max:100, step:5}
R = 0 #@param {type:"slider", min:-40, max:100, step:5}
#@markdown # [Other options:](https://github.com/anothermartz/Easy-Wav2Lip#other-options)
nosmooth = True #@param {type:"boolean"}
output_height = "half resolution (most stable)" #@param ["half resolution (most stable)", "full resolution (may run out of memory)", "480"] {allow-input: true}
#@markdown > input a number or select full / half resolution - default for this github is 480 - which may result in a subtle stretch to the video
resolution_scale = 1
pad_up = str(round(U * resolution_scale))
pad_down = str(round(D * resolution_scale))
pad_left = str(round(L * resolution_scale))
pad_right = str(round(R * resolution_scale))
outfile = '/content/cog-Wav2Lip/results/output.mp4'
if os.path.isfile(outfile):
  os.remove(outfile)

res_custom = False
if output_height == 'half resolution (most stable)':
  resolution_scale = 2
elif output_height == 'full resolution (may run out of memory)':
  resolution_scale = 1
else:
  res_custom = True

in_width, in_height, in_fps, in_length = get_video_details(video_or_image)
out_height = round(in_height / resolution_scale)

if res_custom:
  out_height = int(output_height)
  
#start processing timer
start_time = time.time()

!python inference.py \
--checkpoint_path '/content/cog-Wav2Lip/checkpoints/Wav2Lip.pth' \
--face {video_or_image} --audio {vocal_track} \
--outfile {outfile} \
--out_height $out_height \
--pads $U $D $L $R {'--nosmooth ' if nosmooth else ''}



#end processing timer
end_time = time.time()
elapsed_time = end_time - start_time
process_time = int(elapsed_time)
formatted_process_time = format_time(elapsed_time)

if os.path.isfile(outfile):
  clear_output()
  print(f"Processing time: {formatted_process_time}")
#  print(f'GPU is {gpu_name}')
  show_video(outfile)

Processing time: 1m 27s
